In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
from collections import Counter

# 设置随机种子以保证可重复性
#torch.manual_seed(42)
#np.random.seed(42)

In [2]:
# 1. 加载和准备数据
print("1. 加载鸢尾花数据集...")
iris = load_iris()
X = iris.data  # 特征 (150, 4)
y = iris.target  # 标签 (150,)
feature_names = iris.feature_names
target_names = iris.target_names

print(f"数据集形状: X={X.shape}, y={y.shape}")
print(f"特征: {feature_names}")
print(f"类别: {target_names}")
print(f"类别分布: {Counter(y)}")

1. 加载鸢尾花数据集...
数据集形状: X=(150, 4), y=(150,)
特征: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
类别: ['setosa' 'versicolor' 'virginica']
类别分布: Counter({0: 50, 1: 50, 2: 50})


In [3]:
# 2. 数据预处理
# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 转换为PyTorch张量
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)  # 分类任务使用long类型

In [4]:
# 3. 数据集划分
print("\n2. 划分数据集...")
total_size = len(X_tensor)
train_size = int(0.7 * total_size)  # 70% 训练集
val_size = int(0.15 * total_size)   # 15% 验证集
test_size = total_size - train_size - val_size  # 15% 测试集

# 创建数据集
full_dataset = TensorDataset(X_tensor, y_tensor)

# 随机划分
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size], 
    generator=torch.Generator().manual_seed(42)
)

print(f"训练集大小: {len(train_dataset)}")
print(f"验证集大小: {len(val_dataset)}")
print(f"测试集大小: {len(test_dataset)}")


2. 划分数据集...
训练集大小: 105
验证集大小: 22
测试集大小: 23


In [5]:
# 4. 创建数据加载器
print("\n3. 创建小批量数据加载器...")
batch_size = 16  # 小批量大小

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 显示一批数据的示例
for batch_x, batch_y in train_loader:
    print(f"一批数据的形状: X={batch_x.shape}, y={batch_y.shape}")
    print(f"一批样本的标签: {batch_y.numpy()}")
    break


3. 创建小批量数据加载器...
一批数据的形状: X=torch.Size([16, 4]), y=torch.Size([16])
一批样本的标签: [1 2 2 0 0 1 0 0 0 1 1 2 0 2 0 0]


In [6]:
# 5. 定义神经网络模型
print("\n4. 定义神经网络模型...")
class IrisClassifier(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=10, output_dim=3):
        super(IrisClassifier, self).__init__()
        
        # 定义网络层
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
        # 激活函数
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)  # 添加dropout防止过拟合
        
    def forward(self, x):
        x = self.fc1(x)
        #x = self.relu(self.fc1(x))
        #x = self.dropout(x)
        #x = self.relu(self.fc2(x))
        #x = self.dropout(x)
        #x = self.fc3(x)  # 注意：不应用softmax，因为CrossEntropyLoss会处理
        return x


4. 定义神经网络模型...


In [7]:
# 创建模型实例
model = IrisClassifier(input_dim=4, hidden_dim=10, output_dim=3)
print(model)

# 计算模型参数数量
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"总参数: {total_params}, 可训练参数: {trainable_params}")

IrisClassifier(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)
总参数: 193, 可训练参数: 193


In [8]:
# 6. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 交叉熵损失，适用于多分类
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam优化器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)  # 学习率调度器

In [9]:
# 7. 训练模型
print("\n5. 开始训练模型...")
n_epochs = 20
train_losses = []
val_losses = []
val_accuracies = []
best_val_acc = 0.0
best_model_state = None

for epoch in range(n_epochs):
    # 训练阶段
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_x, batch_y in train_loader:
        # 前向传播
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 统计
        train_loss += loss.item() * batch_x.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += batch_y.size(0)
        train_correct += (predicted == batch_y).sum().item()
    
    avg_train_loss = train_loss / len(train_dataset)
    train_accuracy = 100.0 * train_correct / train_total
    
    # 验证阶段
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            val_loss += loss.item() * batch_x.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += batch_y.size(0)
            val_correct += (predicted == batch_y).sum().item()
    
    avg_val_loss = val_loss / len(val_dataset)
    val_accuracy = 100.0 * val_correct / val_total
    
    # 记录损失和准确率
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)
    
    # 更新学习率
    scheduler.step(avg_val_loss)
    
    # 保存最佳模型
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        best_model_state = model.state_dict().copy()
    
    # 打印进度
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1:3d}/{n_epochs}], '
              f'Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, '
              f'Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%, '
              f'LR: {optimizer.param_groups[0]["lr"]:.6f}')

# 加载最佳模型
model.load_state_dict(best_model_state)
print(f"\n最佳验证准确率: {best_val_acc:.2f}%")




5. 开始训练模型...
Epoch [  5/20], Train Loss: 1.2920, Val Loss: 1.3692, Train Acc: 69.52%, Val Acc: 59.09%, LR: 0.010000
Epoch [ 10/20], Train Loss: 0.7669, Val Loss: 0.8091, Train Acc: 76.19%, Val Acc: 77.27%, LR: 0.010000
Epoch [ 15/20], Train Loss: 0.5479, Val Loss: 0.5803, Train Acc: 88.57%, Val Acc: 86.36%, LR: 0.010000
Epoch [ 20/20], Train Loss: 0.4401, Val Loss: 0.4736, Train Acc: 89.52%, Val Acc: 86.36%, LR: 0.010000

最佳验证准确率: 86.36%


In [10]:
# 8. 评估模型
print("\n6. 在测试集上评估模型...")
model.eval()
test_correct = 0
test_total = 0
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x)
        _, predicted = torch.max(outputs.data, 1)
        test_total += batch_y.size(0)
        test_correct += (predicted == batch_y).sum().item()
        
        # 收集预测结果用于进一步分析
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

test_accuracy = 100.0 * test_correct / test_total
print(f"测试集准确率: {test_accuracy:.2f}%")




6. 在测试集上评估模型...
测试集准确率: 78.26%


In [11]:
# 9. 分类结果评价
print("\n7. 分类结果详细评价...")

# 分类报告
print("\n分类报告:")
print(classification_report(all_labels, all_predictions, target_names=target_names))

# 混淆矩阵
conf_matrix = confusion_matrix(all_labels, all_predictions)
print("混淆矩阵:")
print(conf_matrix)


7. 分类结果详细评价...

分类报告:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         6
  versicolor       0.67      0.33      0.44         6
   virginica       0.71      0.91      0.80        11

    accuracy                           0.78        23
   macro avg       0.79      0.75      0.75        23
weighted avg       0.78      0.78      0.76        23

混淆矩阵:
[[ 6  0  0]
 [ 0  2  4]
 [ 0  1 10]]
